In [52]:
import polars as pl
from pprint import pprint
from typing import List
import xlsx_generator as tablegen

## Pomocné funkce

In [53]:
# Abych nebyl nařčen z plagiátorství, zdroj: https://stackoverflow.com/questions/75954280/how-to-change-the-position-of-a-single-column-in-python-polars-library
def replace(df:pl.DataFrame, new_position:int, col_name:str) -> List[str]:
    """Removes a column at specified position and inserts a named column in its stead.

    Args:
        df (pl.DataFrame): Modified dataframe.
        new_position (int): Position of removed column.
        col_name (str): Name of replacing column.

    Returns:
        List[str]: List of dataframe columns with correct order. Use on a selection to replace the column.
    """
    neworder=df.columns
    neworder.pop(new_position)
    neworder.remove(col_name)
    neworder.insert(new_position,col_name)
    return neworder

In [54]:
def fix_str_to_int(data:pl.DataFrame, fix_list:list) -> pl.DataFrame:
    """Converts columns from a provided list from string to list of ints.

    Args:
        data (pl.DataFrame): Modified dataframe.
        fix_list (list): List of columns to convert.

    Returns:
        pl.DataFrame: Dataframe with fixed columns.
    """

    column_types = data.select(fix_list).dtypes

    # Pokud v žádném sloupci není víc hodnot, polars to převede na int automaticky.
    for index, column in enumerate(fix_list):
        if column_types[index] != pl.String:
            fix_list.remove(column)

    data = data.with_columns(
        pl.col(fix_list)
        .str.split(",")
        .list.eval(
            pl.element()
            .str.strip_chars()
            .cast(pl.Int64, strict=False)
        )
    )

    # Převedení seznamů s pouze None hodnotami na None
    for column in fix_list:
        data = data.with_columns(pl.when(pl.col(column).list.eval(pl.element().is_null()).list.all()).then(None).otherwise(pl.col(column)).alias("bools"))
        data = data.select(replace(data, data.get_column_index(column), "bools")).rename({"bools":column})

    return data


In [55]:
def prep_csv(dataframe:pl.DataFrame) -> pl.DataFrame:
    """Prepares dataframe for export to CSV. This is done by converting lists to string.

    Args:
        dataframe (pl.DataFrame): Prepared dataframe.

    Returns:
        pl.DataFrame: Dataframe with no lists.
    """
    import numpy as np
    type_list = dataframe.dtypes
    mask = np.array([isinstance(column, pl.List) for column in type_list])
    column_list = np.array(dataframe.columns)

    if "identifier" in column_list:
        dataframe = dataframe.drop("identifier")

    list_columns = column_list[mask]
    print(list_columns)

    for column in list_columns:
        dataframe = dataframe.with_columns(pl.col(column).cast(pl.List(pl.Utf8)).list.join(","))
    return dataframe
    

## Načítání

In [56]:
# NOVÉ NAČÍTÁNÍ - Přidat integraci s frontendem, přidat do tablegenu dynamické pojmenování
import xlsx_generator as tablegen

names = tablegen.pull_data(
    search_type="Fakulta",
    search_target="PRF",
    ticket_over="30088f13cc4a64c91aef019587bf2a31f7ff7055306e11abaef001d927dd099a",
    auth_over=["st101885","x0301093100"]
)
print(names.keys())

name_mod = "_" + names["identifier"]

rozvrh_by_kat = pl.read_csv(names["rozvrhy"]).drop("semestr").unique().rename({"predmet" : "zkratka"})
rozvrh_by_kat = fix_str_to_int(rozvrh_by_kat, ["ucitIdno.ucitel", "vsichniUciteleUcitIdno"])

predmety_by_kat = pl.read_csv(names["predmety"]).drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])
predmety_by_kat = fix_str_to_int(predmety_by_kat, ["garantiUcitIdno", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "hodZaSemKombForma"])

Moving to: CPPV
Fetching CSVs.
Fetched CSVs successfully.
One or more CSV's are empty. Continuing to next item.
Moving to: KBI
Fetching CSVs.
Fetched CSVs successfully.
Ensuring type consistency.
Type consistency ensured. Saving testing file and appending the main dataframes.
Dataframes appended. Continuing to next item.
Moving to: KFY
Fetching CSVs.
Fetched CSVs successfully.
Ensuring type consistency.
Type consistency ensured. Saving testing file and appending the main dataframes.
Dataframes appended. Continuing to next item.
Moving to: KGEO
Fetching CSVs.
Fetched CSVs successfully.
Ensuring type consistency.
Type consistency ensured. Saving testing file and appending the main dataframes.
Dataframes appended. Continuing to next item.
Moving to: KCH
Fetching CSVs.
Fetched CSVs successfully.
Ensuring type consistency.
Type consistency ensured. Saving testing file and appending the main dataframes.
Dataframes appended. Continuing to next item.
Moving to: KI
Fetching CSVs.
Fetched CSVs s

In [57]:
# LEGACY NAČÍTÁNÍ

# rozvrh_by_kat = pl.read_csv("source_tables/rozvrhy_fakulta.csv").drop("semestr").unique().rename({"predmet" : "zkratka"})
# rozvrh_by_kat = fix_str_to_int(rozvrh_by_kat, ["ucitIdno.ucitel", "vsichniUciteleUcitIdno"])

# predmety_by_kat = pl.read_csv("source_tables/predmety_fakulta.csv").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])
# predmety_by_kat = fix_str_to_int(predmety_by_kat, ["garantiUcitIdno", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "hodZaSemKombForma"])

In [58]:
print(predmety_by_kat.__len__())
print(rozvrh_by_kat.__len__())
predmety_by_kat.head(7)

3107
886


katedra,zkratka,rok,nazev,nazevDlouhy,maVyuku,vyukaZS,vyukaLS,jakCastoJeNabizen,jakCastoJeNabizenUpresneni,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]
"""CNB""","""BP502""",2024,"""Fyziologie živ…","""Fyziologie živ…","""A""","""A""","""N""","""K""","""""",6,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null
"""CNB""","""EPNS""",2024,"""Elektrospining…","""Elektrospining…","""A""","""A""","""N""","""K""","""""",3,"""NE""","""""","""'RNDr. Petr Ry…","""'RNDr. Petr Ry…",[4587],"""'RNDr. Petr Ry…","""'RNDr. Petr Ry…",[4587],"""'RNDr. Petr Ry…","""'RNDr. Petr Ry…",[4587],"""""","""""",null,"""""","""""","""""","""""","""CNB/UNM, KFY/F…","""""","""""","""'Čapková P., M…","""""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem tohoto k…","""Zkouška""","""ANO""","""Kombinovaná""","""""","""Schema teoreti…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""Bc.""","""Bc.""","""A""",[0]
"""CNB""","""KB502""",2024,"""Fyziologie živ…","""Fyziologie živ…","""A""","""A""","""A""","""K""","""""",5,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'doc. MUDr. Vl…","""'doc. MUDr. Vl…","[281, 5275, 8024]","""'doc. MUDr. Vl…","""'doc. MUDr. Vl…","[281, 5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KBI/KB502""","""""","""""","""N""",12,"""HOD/SEM""",9,"""HOD/SEM""",0,"""HOD/SEM""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",null
"""CNB""","""KB617""",2024,"""Molekulární bi…","""Molekulární bi…","""A""","""N""","""A""","""K""","""""",5,"""NE""","""""","""'Mgr. Michaela…","""'Mgr. Michaela…",[4472],"""'Mgr. Michaela…","""'Mgr. Michaela…",[4472],"""'Mgr. Michaela…","""'Mgr. Michaela…","[4472, 8314]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KBI/KB617""","""""","""""","""N""",8,"""HOD/SEM""",16,"""HOD/SEM""",0,"""HOD/SEM""","""Cílem tohoto k…","""Zkouška""","""ANO""","""Kombinovaná""","""100% účast na …","""1. Centrální d…","""""","""""","""""","""""","""Čeština""",""""""

In [59]:
bez_problemu = predmety_by_kat.select(["katedra", "zkratka", "nazev"]).with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier"))

In [60]:
# testing = pl.read_csv("source_testing/ucitel_predmety_lite").with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier"))
# bez_problemu.join(testing, "identifier", "anti").head()

#predmety_by_kat.select.filter(predmety_by_kat.is_unique().not_())

In [61]:
# qr = ~pl.all().is_duplicated()
# rozvrh_by_kat = rozvrh_by_kat.filter(qr)
# predmety_by_kat = predmety_by_kat.filter(qr)


In [62]:
print(predmety_by_kat.filter(pl.col("katedra") != "KMA").__len__())
predmety_by_kat.filter(pl.col("katedra") != "KMA").head(7)

2782


katedra,zkratka,rok,nazev,nazevDlouhy,maVyuku,vyukaZS,vyukaLS,jakCastoJeNabizen,jakCastoJeNabizenUpresneni,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]
"""CNB""","""BP502""",2024,"""Fyziologie živ…","""Fyziologie živ…","""A""","""A""","""N""","""K""","""""",6,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null
"""CNB""","""EPNS""",2024,"""Elektrospining…","""Elektrospining…","""A""","""A""","""N""","""K""","""""",3,"""NE""","""""","""'RNDr. Petr Ry…","""'RNDr. Petr Ry…",[4587],"""'RNDr. Petr Ry…","""'RNDr. Petr Ry…",[4587],"""'RNDr. Petr Ry…","""'RNDr. Petr Ry…",[4587],"""""","""""",null,"""""","""""","""""","""""","""CNB/UNM, KFY/F…","""""","""""","""'Čapková P., M…","""""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem tohoto k…","""Zkouška""","""ANO""","""Kombinovaná""","""""","""Schema teoreti…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""Bc.""","""Bc.""","""A""",[0]
"""CNB""","""KB502""",2024,"""Fyziologie živ…","""Fyziologie živ…","""A""","""A""","""A""","""K""","""""",5,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'doc. MUDr. Vl…","""'doc. MUDr. Vl…","[281, 5275, 8024]","""'doc. MUDr. Vl…","""'doc. MUDr. Vl…","[281, 5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KBI/KB502""","""""","""""","""N""",12,"""HOD/SEM""",9,"""HOD/SEM""",0,"""HOD/SEM""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",null
"""CNB""","""KB617""",2024,"""Molekulární bi…","""Molekulární bi…","""A""","""N""","""A""","""K""","""""",5,"""NE""","""""","""'Mgr. Michaela…","""'Mgr. Michaela…",[4472],"""'Mgr. Michaela…","""'Mgr. Michaela…",[4472],"""'Mgr. Michaela…","""'Mgr. Michaela…","[4472, 8314]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KBI/KB617""","""""","""""","""N""",8,"""HOD/SEM""",16,"""HOD/SEM""",0,"""HOD/SEM""","""Cílem tohoto k…","""Zkouška""","""ANO""","""Kombinovaná""","""100% účast na …","""1. Centrální d…","""""","""""","""""","""""","""Čeština""",""""""

In [63]:
rozvrh_by_kat.null_count()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,85,118,0,0,0,0,0,0,85,0,0,78,78,0,0,0,0,0,0,0,0,142,142,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,118,0,0,0,0,0,0,0,0


In [64]:
predmety_by_kat.null_count()

katedra,zkratka,rok,nazev,nazevDlouhy,maVyuku,vyukaZS,vyukaLS,jakCastoJeNabizen,jakCastoJeNabizenUpresneni,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,404,0,0,1652,0,0,1294,0,0,2699,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1896


In [65]:
# vybírám pouze předměty, které mají rozvrhovou akci

In [66]:
predmety_s_akci = predmety_by_kat.join(other=rozvrh_by_kat, on="zkratka", how="inner").select(predmety_by_kat.columns).unique().sort("zkratka")
str(len(predmety_s_akci)) + "   " + str(len(predmety_by_kat)) 
predmety_s_akci.filter(pl.col("zkratka") == "P522").head()

katedra,zkratka,rok,nazev,nazevDlouhy,maVyuku,vyukaZS,vyukaLS,jakCastoJeNabizen,jakCastoJeNabizenUpresneni,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]


In [67]:
predmety_s_akci = predmety_s_akci.with_columns(pl.concat_str([pl.col("katedra"), pl.col("zkratka")], separator="/").alias("identifier"))

In [68]:
jednotek_prednasek = pl.col("jednotekPrednasek")
jednotek_cviceni = pl.col("jednotekCviceni")
jednotek_seminare = pl.col("jednotekSeminare")

garant = pl.col("garantiUcitIdno")
cvicici = pl.col("cviciciUcitIdno")
prednasejici = pl.col("prednasejiciUcitIdno")
seminarici = pl.col("seminariciUcitIdno")

# Vyučující a garanti

### Každý předmět má právě jednoho garanta

> Zde jsou předměty, které nemají garanta:

In [69]:
zkratky = predmety_s_akci.filter(garant.is_null()).select(["katedra", "zkratka", "identifier", "rok", "nazev", "nazevDlouhy", "garanti", "garantiSPodily"]).filter(
    # Není SZ
    # pl.col("zkratka").str.starts_with("SZ").is_not()
    True
)
print(f"\n{len(zkratky)} Předmětů nemá zadaného garanta")
bez_problemu = bez_problemu.join(zkratky,"identifier" ,"anti")

prep_csv(zkratky).write_csv("results_csv/bez_garanta"+name_mod+".csv")
# print(list(zkratky), sep=", ")

bez_problemu.head()


57 Předmětů nemá zadaného garanta
[]


katedra,zkratka,nazev,identifier
str,str,str,str
"""CNB""","""BP502""","""Fyziologie živ…","""BP502/CNB"""
"""CNB""","""EPNS""","""Elektrospining…","""EPNS/CNB"""
"""CNB""","""KB502""","""Fyziologie živ…","""KB502/CNB"""
"""CNB""","""KB617""","""Molekulární bi…","""KB617/CNB"""
"""CNB""","""KPCIM""","""Prakt. cvičení…","""KPCIM/CNB"""


> Předměty podezřelé z více garantů:

In [70]:
# If block pokrývá situace kde není žádný předmět s více garanty
if predmety_s_akci.dtypes[predmety_s_akci.get_column_index("garantiUcitIdno")] == pl.List:
    vice_garantu = predmety_s_akci.with_columns(
        garant.list.len().alias("pocet garantu")
        ).select(
            ["garantiUcitIdno", "garanti","katedra", "zkratka", "identifier", "pocet garantu"]
        ).filter(pl.col("pocet garantu") > 1)
    #vice_garantu.write_excel("results_xlsx/vice_garantu.xlsx")
    prep_csv(vice_garantu).write_csv("results_csv/vice_garantu"+name_mod+".csv")
    vice_garantu.head(10)
    bez_problemu = bez_problemu.join(vice_garantu,"identifier" ,"anti")

['garantiUcitIdno']


### Každý předmět má alespoň jednoho vyučujícího v kategoriích dle parametrů předmětu

např. pokud má předmět rozsah 2 h přednášek a 2 h cvičení, pak musí mít alespoň jednoho přednášejícího a alespoň jednoho cvičícího a zároveň nesmí mít vyučujícího v kategorii vede seminář.

In [71]:
# Zvýšená škálovatelnost
def has_teacher_theoretical(dataframe:pl.DataFrame, teacher_type:str) -> pl.DataFrame:
    """Find subjects with no teachers for a specific period type.

    Args:
        dataframe (pl.DataFrame): Dataframe with annotations of subjects.
        teacher_type (str): Teacher for that type of period (eg. prednasejici).

    Returns:
        pl.DataFrame: Dataframe containing offending subjects. Columns: katedra, zkratka, name of all teachers of that type (should be empty), teachers' ids (ditto), how many periods of that type, unit of the former.
    """
    to_period = {
        "prednasejici":"jednotekPrednasek",
        "cvicici":"jednotekCviceni",
        "seminarici":"jednotekSeminare",
    }
    period_type = to_period[teacher_type]

    has_period_type = pl.col(period_type) != 0
    has_no_teacher = pl.col(teacher_type + "UcitIdno").is_null()

    missing_teach = dataframe.filter(has_period_type & has_no_teacher).select("katedra", "zkratka", "identifier", teacher_type, teacher_type + "UcitIdno", period_type, period_type.replace("ek", "ka", 1).replace("ek","ky",1))

    return missing_teach


In [72]:
# Přednášející
chybi_prednasejici = has_teacher_theoretical(predmety_s_akci, "prednasejici")

prep_csv(chybi_prednasejici).write_csv("results_csv/chybi_prednasejici"+name_mod+".csv")
bez_problemu = bez_problemu.join(chybi_prednasejici,"identifier" ,"anti")

['prednasejiciUcitIdno']


In [73]:
# Cvičící
chybi_cvicici = has_teacher_theoretical(predmety_s_akci, "cvicici")

prep_csv(chybi_cvicici).write_csv("results_csv/chybi_cvicici"+name_mod+".csv")
bez_problemu = bez_problemu.join(chybi_cvicici,"identifier" ,"anti")

['cviciciUcitIdno']


In [74]:
# Seminařicí
chybi_seminarici = has_teacher_theoretical(predmety_s_akci, "seminarici")

prep_csv(chybi_cvicici).write_csv("results_csv/chybi_seminarici.xlsx")
bez_problemu = bez_problemu.join(chybi_seminarici,"identifier" ,"anti")

['cviciciUcitIdno']


#### Legacy varianta

In [75]:
# ma_prednasku = jednotek_prednasek != 0
# nema_prednasejiciho = prednasejici.is_null()

# chybi_prednasejici = predmety_s_akci.filter(ma_prednasku & nema_prednasejiciho).select("katedra", "zkratka", "prednasejici", "prednasejiciUcitIdno", "jednotekPrednasek", "jednotkaPrednasky")
# chybi_prednasejici.write_excel("results_xlsx/chybi_prednasejici.xlsx")

# prep_csv(chybi_prednasejici).write_csv("results/chybi_prednasejici"+name_mod+".csv")
# chybi_prednasejici.head()
# #bez_problemu = bez_problemu.join(chybi_prednasejici,"identifier" ,"anti")

In [76]:
# ma_cviceni = jednotek_cviceni != 0
# nema_cviciciho = cvicici.is_null()

# chybi_cvicici = predmety_s_akci.filter(ma_cviceni & nema_cviciciho).select("katedra", "zkratka", "cvicici", "cviciciUcitIdno", "jednotekCviceni", "jednotkaCviceni")
# chybi_cvicici.write_excel("results_xlsx/chybi_cvicici.xlsx")
# prep_csv(chybi_cvicici).write_csv("results_csv/chybi_cvicici"+name_mod+".csv")
# chybi_cvicici.head()
# bez_problemu = bez_problemu.join(chybi_cvicici,"identifier" ,"anti")

In [77]:
# ma_seminare = jednotek_seminare != 0
# nema_seminariciho = seminarici.is_null()

# chybi_seminarici = predmety_s_akci.filter(ma_seminare & nema_seminariciho).select("katedra", "zkratka", "seminarici", "seminariciUcitIdno", "jednotekSeminare", "jednotkaSeminare")
# chybi_seminarici.write_excel("results_xlsx/chybi_seminarici.xlsx")
# prep_csv(chybi_seminarici).write_csv("results_csv/chybi_seminarici.xlsx")
# bez_problemu = bez_problemu.join(chybi_seminarici,"identifier" ,"anti")

> Zde jsou předměty, ve kterých učitel vede seminář:

In [78]:
#prednasejici_je_seminarici = prednasejici == seminarici
#cvicici_je_seminarici = cvicici == seminarici

#konflikt_vyucujicich = predmety_s_akci.filter(ma_seminare & ma_prednasku & (prednasejici_je_seminarici | cvicici_je_seminarici))
#konflikt_vyucujicich.write_excel("results/konflikt_vyucujicich.xlsx")
#konflikt_vyucujicich.head()

In [79]:
print(f"{len(chybi_prednasejici)} předmětů nemá přednášejícího")
print(f"{len(chybi_cvicici)} předmětů nemá cvičícího")
print(f"{len(chybi_seminarici)} předmětů nemá seminářícího")
# print(f"U {len(konflikt_vyucujicich)} předmětů je seminářící také učitel")

8 předmětů nemá přednášejícího
27 předmětů nemá cvičícího
3 předmětů nemá seminářícího


### Garant by měl být někde mezi vyučujícími. Pokud má předmět přednášku, měl by být přednášející.

> V těchto předmětech garant vůbec neučí:

In [80]:
predmety_kgn = predmety_s_akci.filter((jednotek_cviceni != pl.lit(0)) | (jednotek_prednasek != pl.lit(0)) | (jednotek_seminare != pl.lit(0)))
if predmety_kgn.dtypes[predmety_kgn.get_column_index("garantiUcitIdno")] == pl.List:
    predmety_kgn = predmety_kgn.explode("garantiUcitIdno")
predmety_kgn = predmety_kgn.with_columns(
    ((prednasejici.list.contains(garant)) | (cvicici.list.contains(garant)) | (seminarici.list.contains(garant))).alias("containBool")
)

aggreg_kgn = (
    predmety_kgn.lazy().group_by("identifier").agg(
        pl.when(pl.col("containBool") == False).then(garant)
    ).with_columns(garant.list.drop_nulls()).filter(garant.list.len() > 0)
)

selection = ["identifier","katedra", "zkratka", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "jednotekPrednasek", "jednotekCviceni", "jednotekSeminare"]
predmety_kde_garant_neuci = aggreg_kgn.collect().join(predmety_kgn.select(selection), "identifier", "left").drop("identifier")
prep_csv(predmety_kde_garant_neuci).write_csv("results_csv/predmety_kde_garant_neuci"+name_mod+".csv")
predmety_kde_garant_neuci.head()
# bez_problemu = bez_problemu.join(predmety_kde_garant_neuci,"identifier" ,"anti")

['garantiUcitIdno' 'prednasejiciUcitIdno' 'cviciciUcitIdno'
 'seminariciUcitIdno']


garantiUcitIdno,katedra,zkratka,prednasejiciUcitIdno,cviciciUcitIdno,seminariciUcitIdno,jednotekPrednasek,jednotekCviceni,jednotekSeminare
list[i64],str,str,list[i64],list[i64],list[i64],i64,i64,i64
[1983],"""KBI""","""BK201""",null,null,null,6,0,0
[8441],"""KBI""","""KPSPN""",null,null,[8952],0,0,4
[1610],"""KCH""","""KPPP""",null,null,"[8744, 3704]",0,0,13
[8441],"""KBI""","""KPSPP""",null,null,[8952],0,0,3
[69],"""KFY""","""PM""","[1803, 304, … 3521]","[1803, 304, … 3521]",null,2,3,0


> v těchto předmětech je přednáška, ale nedělá jí garant:

In [81]:
# garant_neni_prednasejici = ((garant.list.eval(prednasejici.list.contains(pl.element())))) & (jednotek_prednasek != pl.lit(0)) # & (prednasejici.list.len() != 1)
# garant_neprednasi = predmety_s_akci.filter(garant_neni_prednasejici).select("katedra", "zkratka", "garanti", "garantiUcitIdno", "prednasejici", "prednasejiciUcitIdno")

garant_neprednasi = predmety_s_akci.filter(jednotek_prednasek != pl.lit(0)).explode("garantiUcitIdno").with_columns(
    prednasejici.list.contains(garant).alias("containBool")
)

#print(garant_neprednasi.select("containBool", "garantiUcitIdno").head())

aggregation = (
    garant_neprednasi.lazy().group_by("identifier").agg(
        #pl.col("identifier"),
        pl.when(pl.col("containBool") == False).then(garant)
    ).with_columns(garant.list.drop_nulls()).filter(garant.list.len() > 0)
)

garant_neprednasi_post = aggregation.collect().join(garant_neprednasi.select("katedra", "zkratka","prednasejiciUcitIdno", "identifier", "jednotekPrednasek"), "identifier", "left").drop("identifier")



# garant_neprednasi.write_excel("results_xlsx/garant_neprednasi.xlsx")
garant_neprednasi_csv = prep_csv(garant_neprednasi_post)
garant_neprednasi_csv.write_csv("results_csv/garant_neprednasi"+name_mod+".csv")
print(garant_neprednasi_post.__len__())
# bez_problemu = bez_problemu.join(garant_neprednasi,"identifier" ,"anti")

garant_neprednasi_post.head()

['garantiUcitIdno' 'prednasejiciUcitIdno']
18


garantiUcitIdno,katedra,zkratka,prednasejiciUcitIdno,jednotekPrednasek
list[i64],str,str,list[i64],i64
[8512],"""KBI""","""BK333""",[930],8
[1644],"""KGEO""","""M107""",[5457],2
[312],"""KGEO""","""B406""",[7684],1
[3553],"""CNB""","""PTVN""",[4587],1
[615],"""KFY""","""M303""",[79],2


## Kontrola proti rozvrhovaným akcím

### Vyučující u rozvrhových akcí musí být v příslušných kategoriích u vyučujících

#### Osekám dataframy

In [82]:
rozvrh_by_kat.head()
# predmety_by_kat.head()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,str,str,i64,i64,str,str,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
495360,"""Výp. a odhad. …","""CNB""","""NA19""","""""","""332""",[332],"""Zdeňka""","""Kolská""","""prof. Ing.""","""Ph.D.""","""A""","""A""","""100""",2024,"""CP""","""2.33""","""8""",9,0,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""5""","""5""",1,"""11:00""","""11:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.N NANO,1.NAN…","""UJEP""","""""","""4.10.2024""","""3.1.2025""","""R""",[332],"""prof. Ing. Zde…","""'prof. Ing. Zd…","""Kolská""",495360,"""""","""""","""RSMEJKAL""",""""""
495361,"""Výp. a odhad. …","""CNB""","""NA19""","""""","""332""",[332],"""Zdeňka""","""Kolská""","""prof. Ing.""","""Ph.D.""","""A""","""A""","""100""",2024,"""CP""","""2.33""","""8""",9,0,"""Cvičení""","""Cv""","""A""","""Pátek""","""Pá""","""""","""6""","""6""",1,"""12:00""","""12:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.N NANO,1.NAN…","""UJEP""","""""","""4.10.2024""","""3.1.2025""","""R""",[332],"""prof. Ing. Zde…","""'prof. Ing. Zd…","""Kolská""",495361,"""""","""""","""RSMEJKAL""",""""""
495357,"""Výp. a odhad. …","""CNB""","""METD""","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""",2024,"""""","""""","""""",777,0,"""Přednáška""","""Př""","""A""","""""","""""","""""","""""","""""",1,"""00:00""","""00:00""",0,0,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""""","""""","""""","""""","""""","""R""",null,"""""","""""","""""",495357,"""""","""""","""RSMEJKAL""",""""""
492230,"""Obecná botanik…","""KBI""","""BP103""","""""","""5969""",[5969],"""Lukáš""","""Bystrianský""","""Mgr.""","""Ph.D.""","""A""","""A""","""100""",2024,"""ZV""","""202""","""30""",17,0,"""Cvičení""","""Cv""","""A""","""Pondělí""","""Po""","""""","""3""","""6""",4,"""09:00""","""12:50""",41,51,"""Lichý""","""L""","""""","""N""","""N""","""Automaticky vy…","""1.BI JO""","""UJEP""","""""","""7.10.2024""","""16.12.2024""","""R""",[5969],"""Mgr. Lukáš Bys…","""'Mgr. Lukáš By…","""Bystrianský""",492230,"""""","""""","""RBYSTRIANSKY""",""""""
492315,"""Plant Biotechn…","""KBI""","""M302""","""""","""5471""",[5471],"""Hana""","""Auer Malinská""","""Mgr.""","""Ph.D.""","""A""","""A""","""100""",2024,"""ZV""","""B2""","""12""",8,0,"""Přednáška""","""Př""","""A""","""Pondělí""","""Po""","""""","""4""","""5""",2,"""10:00""","""11:50""",41,51,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.BI N AB,2.BI…","""UJEP""","""""","""7.10.2024""","""16.12.2024""","""R""",[5471],"""Mgr. Hana Auer…","""'Mgr. Hana Aue…","""Auer Malinská""",492315,"""""","""""","""RBYSTRIANSKY""",""""""


In [83]:
# PROZATÍMNÍ NASTAVENÍ FILTRUJE AKCE BEZ DATUMU

maly_rozvrh = rozvrh_by_kat.select(["katedra","zkratka", "vsichniUciteleUcitIdno", "typAkceZkr", "rok", "datumOd", "datumDo", "hodinaSkutOd", "hodinaSkutDo"]).with_columns(pl.concat_str(pl.col("zkratka"), pl.col("katedra")).alias("identifier"))
maly_rozvrh = maly_rozvrh.with_columns(pl.col("vsichniUciteleUcitIdno")).explode("vsichniUciteleUcitIdno").rename({"vsichniUciteleUcitIdno": "idno"}).filter(pl.col("datumOd").str.len_chars() > 0)
maly_rozvrh.filter(pl.col("katedra").is_null()).head()

katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,identifier
str,str,i64,str,i64,str,str,str,str,str


In [84]:
# predmety_by_kat

In [85]:
from pprint import pprint
# pprint(predmety_by_kat.columns,compact=True)
predmety_by_kat.head()

katedra,zkratka,rok,nazev,nazevDlouhy,maVyuku,vyukaZS,vyukaLS,jakCastoJeNabizen,jakCastoJeNabizenUpresneni,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]
"""CNB""","""BP502""",2024,"""Fyziologie živ…","""Fyziologie živ…","""A""","""A""","""N""","""K""","""""",6,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null
"""CNB""","""EPNS""",2024,"""Elektrospining…","""Elektrospining…","""A""","""A""","""N""","""K""","""""",3,"""NE""","""""","""'RNDr. Petr Ry…","""'RNDr. Petr Ry…",[4587],"""'RNDr. Petr Ry…","""'RNDr. Petr Ry…",[4587],"""'RNDr. Petr Ry…","""'RNDr. Petr Ry…",[4587],"""""","""""",null,"""""","""""","""""","""""","""CNB/UNM, KFY/F…","""""","""""","""'Čapková P., M…","""""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem tohoto k…","""Zkouška""","""ANO""","""Kombinovaná""","""""","""Schema teoreti…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""Bc.""","""Bc.""","""A""",[0]
"""CNB""","""KB502""",2024,"""Fyziologie živ…","""Fyziologie živ…","""A""","""A""","""A""","""K""","""""",5,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'doc. MUDr. Vl…","""'doc. MUDr. Vl…","[281, 5275, 8024]","""'doc. MUDr. Vl…","""'doc. MUDr. Vl…","[281, 5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KBI/KB502""","""""","""""","""N""",12,"""HOD/SEM""",9,"""HOD/SEM""",0,"""HOD/SEM""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",null
"""CNB""","""KB617""",2024,"""Molekulární bi…","""Molekulární bi…","""A""","""N""","""A""","""K""","""""",5,"""NE""","""""","""'Mgr. Michaela…","""'Mgr. Michaela…",[4472],"""'Mgr. Michaela…","""'Mgr. Michaela…",[4472],"""'Mgr. Michaela…","""'Mgr. Michaela…","[4472, 8314]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KBI/KB617""","""""","""""","""N""",8,"""HOD/SEM""",16,"""HOD/SEM""",0,"""HOD/SEM""","""Cílem tohoto k…","""Zkouška""","""ANO""","""Kombinovaná""","""100% účast na …","""1. Centrální d…","""""","""""","""""","""""","""Čeština""",""""""

In [87]:
ciselnik_ucitelu = pl.read_csv("source_tables/ciselnik_ucitelu.csv").select("nazev", "key").rename({"nazev":"jmena", "key":"idno"})
ciselnik_ucitelu.head()

jmena,idno
str,i64
"""Abrahamová Pet…",8409
"""Adamec Slavomí…",5432
"""Adámek Pavel, …",7443
"""Adamík Petr, M…",9822
"""Adam Julie, M.…",9249


In [88]:
male_predmety = predmety_by_kat.with_columns(pl.concat_str(pl.col("zkratka"), pl.col("katedra")).alias("identifier")).select(pl.col(["zkratka", "katedra", "identifier", "rok", "nazev", "garantiUcitIdno", "prednasejici", "prednasejiciUcitIdno","cvicici", "cviciciUcitIdno","seminarici", "seminariciUcitIdno"])).unique(subset="identifier").drop("identifier")
#male_predmety = male_predmety.with_columns(pl.col(["prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "garantiUcitIdno"]).str.split(", "))
male_predmety.filter(pl.col("zkratka") == "B317").head()

zkratka,katedra,rok,nazev,garantiUcitIdno,prednasejici,prednasejiciUcitIdno,cvicici,cviciciUcitIdno,seminarici,seminariciUcitIdno
str,str,i64,str,list[i64],str,list[i64],str,list[i64],str,list[i64]
"""B317""","""KCH""",2024,"""Membránové sep…",[3705],"""'Ing. Zuzana P…",[3705],"""""",null,"""""",null


In [89]:
# přednášející nemá přednášku:
filtrovani_prednasejici = male_predmety.select("nazev", "zkratka", "prednasejiciUcitIdno").explode("prednasejiciUcitIdno")
prednasejici_jmena = male_predmety.select("prednasejici").rename({"prednasejici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
prednasejici_jmena = prednasejici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_prednasejici = filtrovani_prednasejici.with_columns(prednasejici_jmena).filter(
    pl.col("prednasejiciUcitIdno"
).is_not_null()).with_columns(
    prednasejici.alias("idno")
)
joined_prednasejici = filtrovani_prednasejici.join(maly_rozvrh, "idno", "left")
prednasejici_bez_prednasek = joined_prednasejici.filter(pl.col("typAkceZkr").is_null())
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_excel("results_xlsx/prednasejici_bez_prednasek.xlsx")
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_csv("results_csv/prednasejici_bez_prednasek"+name_mod+".csv")
bez_problemu = bez_problemu.join(prednasejici_bez_prednasek,"identifier" ,"anti")

In [90]:
#WEIRD STUFF: Učitel 8027 u KGEO/B508
# otherwise looks pretty good

# Přednáška nemá přednášejícího:

male_prednasky = maly_rozvrh.filter(pl.col("typAkceZkr") == "Př")
joined_prednasky = male_prednasky.join(male_predmety.select("zkratka", "katedra", "nazev", "prednasejici", "prednasejiciUcitIdno"), "zkratka", "left").unique()
prednasky_bez_prednasejicich = joined_prednasky.filter(pl.col("idno").is_in(prednasejici).not_() & ((pl.col("katedra") == pl.col("katedra_right")) | pl.col("katedra_right").is_null()))

prednasky_bez_prednasejicich = prednasky_bez_prednasejicich.join(ciselnik_ucitelu, "idno", "left").with_columns(prednasejici.cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

prednasky_bez_prednasejicich.write_csv("results_csv/prednasky_bez_prednasejicich"+name_mod+".csv")
prednasky_bez_prednasejicich.write_excel("results_xlsx/prednasky_bez_prednasejicich.xlsx")

bez_problemu = bez_problemu.join(prednasky_bez_prednasejicich,"identifier" ,"anti")

print(prednasky_bez_prednasejicich.__len__())
prednasky_bez_prednasejicich.head(15)

39


katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,identifier,katedra_right,nazev,prednasejici,prednasejiciUcitIdno,jmena
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str
"""KPRF""","""101""",556,"""Př""",2024,"""7.10.2024""","""16.12.2024""","""16:00""","""17:50""","""101KPRF""",null,null,null,null,"""Novák Martin, …"
"""KCH""","""B100""",3705,"""Př""",2024,"""1.10.2024""","""17.12.2024""","""15:00""","""16:50""","""B100KCH""","""KCH""","""Úvod do chemie…","""'doc. RNDr. Ja…","""2855""","""Petrusová Zuza…"
"""KGEO""","""B104""",4303,"""Př""",2024,"""1.10.2024""","""17.12.2024""","""13:00""","""14:50""","""B104KGEO""","""KGEO""","""Geografická ka…","""'doc. PhDr. RN…","""3452""","""Dolejš Martin,…"
"""KGEO""","""B104""",5855,"""Př""",2024,"""1.10.2024""","""17.12.2024""","""13:00""","""14:50""","""B104KGEO""","""KGEO""","""Geografická ka…","""'doc. PhDr. RN…","""3452""","""Meyer Petr, Mg…"
"""KGEO""","""B107""",4398,"""Př""",2024,"""7.10.2024""","""16.12.2024""","""14:00""","""15:50""","""B107KGEO""","""KGEO""","""Fyzická geogra…","""'RNDr. Ivan Fa…","""315, 3164""","""Vágner Jiří, R…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""KGEO""","""B309""",828,"""Př""",2024,"""4.10.2024""","""3.1.2025""","""10:00""","""11:50""","""B309KGEO""","""KGEO""","""EU a střední E…","""'doc. RNDr. Ji…","""312""","""Martínek Jiří,…"
"""KGEO""","""B508""",4398,"""Př""",2024,"""7.10.2024""","""16.12.2024""","""10:00""","""11:50""","""B508KGEO""","""KGEO""","""Ochrana přírod…","""'RNDr. Tomáš M…","""1472""","""Vágner Jiří, R…"
"""KCH""","""BK100""",3705,"""Př""",2024,"""12.10.2024""","""12.10.2024""","""09:00""","""12:50""","""BK100KCH""","""KCH""","""Úvod do chemie…","""'doc. RNDr. Ja…","""2855""","""Petrusová Zuza…"


In [91]:
# cvičící nemá cvičení:
filtrovani_cvicici = male_predmety.select("nazev", "zkratka", "cviciciUcitIdno").explode("cviciciUcitIdno")
cvicici_jmena = male_predmety.select("cvicici").rename({"cvicici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
cvicici_jmena = cvicici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_cvicici = filtrovani_cvicici.with_columns(cvicici_jmena).filter(
    pl.col("cviciciUcitIdno"
).is_not_null()).with_columns(
    cvicici.alias("idno")
)
joined_cvicici = filtrovani_cvicici.join(maly_rozvrh, "idno", "left")
cvicici_bez_cviceni = joined_cvicici.filter(pl.col("typAkceZkr").is_null())
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno").sort("cviciciUcitIdno").write_excel("results_xlsx/cvicici_bez_cviceni.xlsx")
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno", "jmena").sort("cviciciUcitIdno").write_csv("results_csv/cvicici_bez_cviceni"+name_mod+".csv")

bez_problemu = bez_problemu.join(cvicici_bez_cviceni,"identifier" ,"anti")

In [92]:
#WEIRD STUFF: KGEO/0165 má napsaná cvičení (jak v rozvrhu tak předmětu) ale nemá napsanýho žádnýho učitele (fakt ani garanta, ani jednoho přednášejícího/cvičícího/etc.), tf?

male_cviceni = maly_rozvrh.filter(pl.col("typAkceZkr") == "Cv")
joined_cviceni = male_cviceni.join(male_predmety.select("zkratka", "katedra", "nazev", "cvicici", "cviciciUcitIdno"), "zkratka", "left").unique()
cviceni_bez_cvicich = joined_cviceni.filter(pl.col("idno").is_in(cvicici).not_() & ((pl.col("katedra") == pl.col("katedra_right")) | pl.col("katedra_right").is_null()))

cviceni_bez_cvicich = cviceni_bez_cvicich.join(ciselnik_ucitelu, "idno", "left").with_columns(cvicici.cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

cviceni_bez_cvicich.write_csv("results_csv/cviceni_bez_cvicich"+name_mod+".csv")
cviceni_bez_cvicich.write_excel("results_xlsx/cviceni_bez_cvicich.xlsx")

bez_problemu = bez_problemu.join(cviceni_bez_cvicich,"identifier" ,"anti")

print(cviceni_bez_cvicich.__len__())
cviceni_bez_cvicich.head(15)

79


katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,identifier,katedra_right,nazev,cvicici,cviciciUcitIdno,jmena
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str
"""KCH""","""B100""",3705,"""Cv""",2024,"""2.10.2024""","""18.12.2024""","""10:00""","""11:50""","""B100KCH""","""KCH""","""Úvod do chemie…","""'doc. RNDr. Ja…","""2855""","""Petrusová Zuza…"
"""KCH""","""B100""",3705,"""Cv""",2024,"""2.10.2024""","""18.12.2024""","""08:00""","""09:50""","""B100KCH""","""KCH""","""Úvod do chemie…","""'doc. RNDr. Ja…","""2855""","""Petrusová Zuza…"
"""KCH""","""B103""",1319,"""Cv""",2024,"""14.11.2024""","""14.11.2024""","""15:00""","""17:50""","""B103KCH""","""KCH""","""Bezpečnost a h…","""'MUDr. Eva Sol…","""1800""","""Stupák Rudolf,…"
"""KCH""","""B103""",1319,"""Cv""",2024,"""28.11.2024""","""28.11.2024""","""15:00""","""17:50""","""B103KCH""","""KCH""","""Bezpečnost a h…","""'MUDr. Eva Sol…","""1800""","""Stupák Rudolf,…"
"""KGEO""","""B104""",8819,"""Cv""",2024,"""3.10.2024""","""2.1.2025""","""15:00""","""16:50""","""B104KGEO""","""KGEO""","""Geografická ka…","""'doc. PhDr. RN…","""3452, 5855""","""Bartůněk Marti…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""KGEO""","""B301""",8647,"""Cv""",2024,"""1.10.2024""","""17.12.2024""","""14:00""","""14:50""","""B301KGEO""","""KGEO""","""Geografie Česk…","""'Mgr. Vladan H…","""4222""","""Šmíd Ondřej, M…"
"""KGEO""","""B301""",8647,"""Cv""",2024,"""1.10.2024""","""17.12.2024""","""17:00""","""17:50""","""B301KGEO""","""KGEO""","""Geografie Česk…","""'Mgr. Vladan H…","""4222""","""Šmíd Ondřej, M…"
"""KGEO""","""B304""",9265,"""Cv""",2024,"""3.10.2024""","""2.1.2025""","""12:00""","""12:50""","""B304KGEO""","""KGEO""","""Rozvojové prob…","""""",null,"""Král Tomáš, Mg…"


In [93]:
# seminařicí nemá seminář:
filtrovani_seminarici = male_predmety.select("nazev", "zkratka", "seminariciUcitIdno").explode("seminariciUcitIdno")
seminarici_jmena = male_predmety.select("seminarici").rename({"seminarici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
seminarici_jmena = seminarici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_seminarici = filtrovani_seminarici.with_columns(seminarici_jmena).filter(
    pl.col("seminariciUcitIdno"
).is_not_null()).with_columns(
    seminarici.alias("idno")
)
joined_seminarici = filtrovani_seminarici.join(maly_rozvrh, "idno", "left")
seminarici_bez_seminare = joined_seminarici.filter(pl.col("typAkceZkr").is_null())
seminarici_bez_seminare.select("nazev", "zkratka", "seminariciUcitIdno").sort("seminariciUcitIdno").write_csv("results_csv/seminarici_bez_seminare"+name_mod+".csv")
seminarici_bez_seminare.select("nazev", "zkratka", "seminariciUcitIdno").sort("seminariciUcitIdno").write_excel("results_xlsx/seminarici_bez_seminare.xlsx")

bez_problemu = bez_problemu.join(seminarici_bez_seminare,"identifier" ,"anti")

In [94]:
male_seminare = maly_rozvrh.filter(pl.col("typAkceZkr") == "Se")
joined_seminare = male_seminare.join(male_predmety.select("zkratka", "katedra", "nazev", "seminarici", "seminariciUcitIdno"), "zkratka", "left").unique()
seminare_bez_seminaricich = joined_seminare.filter(pl.col("idno").is_in(seminarici).not_() & ((pl.col("katedra") == pl.col("katedra_right")) | pl.col("katedra_right").is_null()))

seminare_bez_seminaricich = seminare_bez_seminaricich.join(ciselnik_ucitelu, "idno", "left").with_columns(seminarici.cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

seminare_bez_seminaricich.write_csv("results_csv/seminare_bez_seminaricich"+name_mod+".csv")
seminare_bez_seminaricich.write_excel("results_xlsx/seminare_bez_seminaricich.xlsx")

bez_problemu = bez_problemu.join(seminare_bez_seminaricich,"identifier" ,"anti")
print(seminare_bez_seminaricich.__len__())
seminare_bez_seminaricich.head(6)

6


katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,identifier,katedra_right,nazev,seminarici,seminariciUcitIdno,jmena
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str
"""KCH""","""BK308""",4226,"""Se""",2024,"""20.12.2024""","""20.12.2024""","""09:00""","""12:50""","""BK308KCH""","""KCH""","""Odborný seminá…","""'doc. Ing. Jan…","""1064""","""Kadlečková Iva…"
"""KCH""","""BK320""",9475,"""Se""",2024,"""6.12.2024""","""6.12.2024""","""10:00""","""13:50""","""BK320KCH""","""KCH""","""Seminář z prům…","""'doc. Ing. Jar…","""1543""","""Šimek Josef, I…"
"""KBI""","""KRSP""",8952,"""Se""",2024,"""16.11.2024""","""16.11.2024""","""09:00""","""12:50""","""KRSPKBI""","""KBI""","""Komplexní refl…","""'doc. PhDr. Ka…","""8441""","""Svobodová Silv…"
"""KBI""","""KRSP""",8952,"""Se""",2024,"""14.12.2024""","""14.12.2024""","""09:00""","""12:50""","""KRSPKBI""","""KBI""","""Komplexní refl…","""'doc. PhDr. Ka…","""8441""","""Svobodová Silv…"
"""KBI""","""KRSP""",8952,"""Se""",2024,"""2.11.2024""","""2.11.2024""","""09:00""","""12:50""","""KRSPKBI""","""KBI""","""Komplexní refl…","""'doc. PhDr. Ka…","""8441""","""Svobodová Silv…"
"""KFY""","""PPP""",301,"""Se""",2024,"""4.10.2024""","""3.1.2025""","""09:00""","""11:50""","""PPPKFY""","""KFY""","""Průběžná pedag…","""'RNDr. Jiří Kr…","""303""","""Hejnová Eva, R…"


In [ ]:
predmety_by_kat.filter(pl.col("zkratka") == "KTZI2").head()

katedra,zkratka,rok,nazev,nazevDlouhy,maVyuku,vyukaZS,vyukaLS,jakCastoJeNabizen,jakCastoJeNabizenUpresneni,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]
"""KI""","""KTZI2""",2024,"""Teoretické zák…","""Teoretické zák…","""A""","""N""","""A""","""K""","""""",3,"""NE""","""""","""'RNDr. Martin …","""'RNDr. Martin …",[264],"""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""""","""""",null,"""""","""""","""""","""""","""KI/KTZI1""","""""","""""","""'Demel, J. Gra…","""""","""""","""""","""N""",6,"""HOD/SEM""",4,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz je koncip…","""Zápočet""","""NE""","""Kombinovaná""","""Požadavky k zá…","""1. Rekurence a…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0]


#### Vypsání ne-problémů

In [95]:
# bez_problemu = bez_problemu.drop("identifier")
bez_problemu.write_csv("results_csv/bez_problemu"+name_mod+".csv")
bez_problemu.write_excel("results_xlsx/bez_problemu.xlsx")
print(bez_problemu.__len__())
bez_problemu.head()

3107


katedra,zkratka,nazev,identifier
str,str,str,str
"""CNB""","""BP502""","""Fyziologie živ…","""BP502/CNB"""
"""CNB""","""EPNS""","""Elektrospining…","""EPNS/CNB"""
"""CNB""","""KB502""","""Fyziologie živ…","""KB502/CNB"""
"""CNB""","""KB617""","""Molekulární bi…","""KB617/CNB"""
"""CNB""","""KPCIM""","""Prakt. cvičení…","""KPCIM/CNB"""


### warning, když je mezi vyučujícími někdo bez odpovídajícího typu rozvrhové akce

Pro každého učitele vypíšu předměty, u kterých má mít nějakou akci.

Takže budu mít dataframe, kde bude učitel, zkratka, a jeden sloupeček za každý typ akce, kde učitel bude muset mít aspoň 1 narozvrhovaný typ akce s daným předmětem, kde je hodnota `True`


In [ ]:
# vypíšu typy akcí
list(maly_rozvrh.select(pl.col("typAkceZkr").unique()))

[shape: (3,)
 Series: 'typAkceZkr' [str]
 [
 	"Se"
 	"Cv"
 	"Př"
 ]]

### Změny proti akreditacím.

### Porovnat revize předmětů

tj. parametry předmětů pro aktuální akademický rok a pro příští.

> jakou to má mít podobu?